# AlexNet — Architektúra

Súbor: alexnet_0.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(23746, 128, 128, 1)
(23746, 128, 128, 1)


In [6]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

CNN

In [8]:
model = Sequential()

In [9]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [10]:
model.add(Conv2D(96, 11, strides=4, activation='relu')) 

In [11]:
model.add(MaxPooling2D(3, 2))

In [12]:
model.add(Conv2D(256, 5, strides=1, padding='same', activation='relu'))

In [13]:
model.add(MaxPooling2D(3, 2))

In [14]:
model.add(Conv2D(384, 3, strides=1, padding='same', activation='relu'))

In [15]:
model.add(Conv2D(256, 3, strides=1, padding='same', activation='relu'))

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(4096, activation='relu')) 

In [19]:
model.add(Dropout(0.5))

In [20]:
model.add(Dense(4096, activation='relu'))

In [21]:
model.add(Dropout(0.5))

In [22]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [26]:
adam = Adam()

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [28]:
saved_model = "alexnet_0.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [29]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 23746 samples, validate on 23746 samples
Epoch 1/15
23746/23746 [==============================] - 610s 26ms/step - loss: 0.3914 - accuracy: 0.8125 - val_loss: 0.2688 - val_accuracy: 0.9017

Epoch 00001: val_accuracy improved from -inf to 0.90171, saving model to alexnet_0.hdf5
Epoch 2/15
23746/23746 [==============================] - 611s 26ms/step - loss: 0.2416 - accuracy: 0.9085 - val_loss: 0.1909 - val_accuracy: 0.9334

Epoch 00002: val_accuracy improved from 0.90171 to 0.93342, saving model to alexnet_0.hdf5
Epoch 3/15
23746/23746 [==============================] - 557s 23ms/step - loss: 0.2010 - accuracy: 0.9250 - val_loss: 0.1792 - val_accuracy: 0.9397

Epoch 00003: val_accuracy improved from 0.93342 to 0.93970, saving model to alexnet_0.hdf5
Epoch 4/15
23746/23746 [==============================] - 543s 23ms/step - loss: 0.1636 - accuracy: 0.9426 - val_loss: 0.1499 - val_accuracy: 0.9524

Epoch 00004: val_accuracy improved from 0.93970 to 0.95237, saving model to ale

Načítanie modelu

In [7]:
print("Loading model....")
model = load_model('alexnet_0.hdf5')

Loading model....




Vyhodnotenie test loss a accuracy

In [8]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.06314648669824673
Test accuracy: 0.9793649315834045


In [9]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.136359243849736
Test accuracy: 0.966562807559967


Predikcia

In [33]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [34]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [35]:
print(matrix)
# TN / FN
# FP / TP

[[11756   117]
 [  677 11196]]


Classification Report

In [36]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     11873
           1       0.99      0.94      0.97     11873

   micro avg       0.97      0.97      0.97     23746
   macro avg       0.97      0.97      0.97     23746
weighted avg       0.97      0.97      0.97     23746
 samples avg       0.97      0.97      0.97     23746



In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)         885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 256)        